In [23]:
import numpy as np
import random

def deriv(f, x, h=0.0001):
    return (f(x+h) - f(x-h))/(2*h)

def s_deriv(f, x, h=0.0001):
    return (f(x+h) - 2.0*f(x) + f(x-h))/(h*h)

def parabolic_iteration(f, l, r, e=1.0e-20):
    count = 0
    u = l
    v = (r-l)/2.0
    w = r

    while abs(u-w) > e:
        fu = f(u)
        fv = f(v)
        fw = f(w)
        p = (v - u)**2 * (fv - fw) - (v - w)**2 * (fv - fu)
        q = (v - u) * (fv - fw) - (v - w) * (fv - fu)
        if q == 0:
            break
        next = v - p/(2*q)
        
        u = w
        w = v
        v = next

        count+=1

    return v, count

def netwon_optimization(f, l, r, e=1.0e-20):
    count = 0
    x = (r-l)/2.0
    step = 1.0
    while abs(step) > e:
        step = deriv(f,x)/s_deriv(f, x)
        x -= step
        count+=1
    return x, count



The above functions perform optimization in 1 dimension in two different ways, using Newtons method and using interpolation by a parabola and taking its minima to obtain new esimates. Lets test the number of steps needed to obtain a certain accuracy for each method

In [25]:
def f(x):
    return np.sin(x)

def f2(x):
    return -x*x +x

def f3(x):
    return -x*x*x + x*x + 2.0*x

print('(solution, iterations)')
print(netwon_optimization(f, 1, 4))
print(parabolic_iteration(f, 1, 4))
print(netwon_optimization(f2, 0, 3))
print(parabolic_iteration(f2, 0, 3))
print(netwon_optimization(f3, 0, 3))
print(parabolic_iteration(f3, 0, 3))

(solution, iterations)
(1.570796326794995, 4)
(1.5707963261525917, 7)
(0.4999999999999998, 3)
(0.5, 2)
(1.2152504351316875, 5)
(1.2152504394041972, 9)


Newtons method appears to be quicker in general, of course this comes at the cost of extra error and computation needed for computing the derivative. Both of these methods may also fail to converge in some instances, and newtons method may instead converge to an inflection point...

In [ ]:
# generate some examples of failures to converge

There are methods that have guarenteed convergence but this generally comes at the cost of a slower convergence rate. Below is an example of this know as the golden section method

In [ ]:
# implement golden section method

We can combine techniques to achieve guaranteed convergence while hopefully increasing the speed substantially. To do this we simply run the golden section method to some tolerence level then apply newtons method

In [ ]:
# combined method

We can now make this more robust so it actually verifies that the obtained point is a minima and refines or retries if needed

In [ ]:
# finalized method that works in a much wider range of cases

# tests to demonstrate robustness of the final function